<a href="https://colab.research.google.com/github/tjturnage/colab-dash/blob/main/xmacis_mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2><b>Step 1</b></h2>
<h3>Install the packages you need. You can ignore the comments that appears as long as there are no error messages.</h3>

In [ ]:
!pip install jupyter-dash -q
!pip install dash-bootstrap-components
import warnings
warnings.filterwarnings('ignore')

<h2><b>Step 2</b></h2>
<h3>Determine a station you want to get data for. Detailed information about ACIS Web Services <a href="https://www.rcc-acis.org/docs_webservices.html" target="_blank">HERE</a></h3>


In [18]:
import json
import requests
meta_url = "http://data.rcc-acis.org/StnMeta"

#@title
station_name = "GRRthr" #@param ["GRRthr"] {allow-input: true}

try:
    params = {"sids": station_name, "elems": "maxt", "meta": ["name","valid_daterange"]}
    req = requests.post(meta_url,json=params)
    data = req.json()
    meta = data['meta'][0]
    name = meta['name']
    print(f'The station you chose is: {name}.')
except Exception:
  print("Station Not Found! Check your entry and try again.")


The station you chose is: Grand Rapids Area.


<h2><b>Step 3</b></h2>
<h3>Run the code below to retrieve data</h3>


In [19]:
from datetime import datetime
import pandas as pd
import numpy as np

parms_url = "http://data.rcc-acis.org/StnData?params="
meta_url = "http://data.rcc-acis.org/StnMeta"

class XMACIS():
    def __init__(self,station):

        self.station = station

        #grab metadata related to station's Period Of Record (POR)
        self.por_metadata = self.get_por_metadata()
        self.df = self.get_all_por_data()


    def get_por_metadata(self):
        """
        Returns list of strings: ['YYYY-mm-dd','YYYY-mm-dd']
        Refers to first/last dates ofperiod of record
        """
        params = {"sids": self.station, "elems": "maxt", "meta": ["name","valid_daterange"]}
        req = requests.post(meta_url,json=params)
        data = req.json()
        meta = data['meta'][0]
        self.full_name = meta['name']
        #self.por_start = meta['valid_daterange'][0][0]
        self.por_start = '1970-01-01'
        self.por_end = meta['valid_daterange'][0][1]
        self.por_end = '2023-2-20'
        self.por_start_year = int(self.por_start[0:4])
        self.por_end_year = int(self.por_end[0:4])
        self.por_year_range = self.por_end_year - self.por_start_year + 1
        self.por_year_index = pd.date_range(start=f'1/1/{self.por_start_year}', periods=self.por_year_range, freq='Y')
        self.por_label = f'{self.por_start_year} - {self.por_end_year}'
        return

    def get_all_por_data(self):
        parms = {
        "sid":self.station,
        "sdate":self.por_start,
        "edate":self.por_end,
        "elems":[
            {"name":"maxt","interval":"dly","duration":"dly"},
            {"name":"mint","interval":"dly","duration":"dly"},
            {"name":"avgt","interval":"dly","duration":"dly"},
            {"name":"maxt","interval":"dly","duration":"dly","normal":"1"},
            {"name":"mint","interval":"dly","duration":"dly","normal":"1"},
            {"name":"avgt","interval":"dly","duration":"dly","normal":"1"},
            ]}

        req = requests.post(parms_url,json=parms)
        data = req.json()
        df_init = pd.json_normalize(data)
        df_dp = df_init["data"].apply(pd.Series).T
        df = df_dp[0].apply(pd.Series)
        df.columns = ['dts','maxt','mint','avgt','maxt_n','mint_n','avgt_n']
        df['date'] = pd.to_datetime(df['dts'], errors='coerce')
        #df['date'] = pd.DatetimeIndex(df['dts'])
        df.set_index('date', inplace=True)
        df['year'] = df.index.year
        df['month'] = df.index.month
        df['day'] = df.index.day
        df = df.astype({"dts":'string', "year":'int', "month":'int', "day":'int', "maxt":'int', "mint":'int', "avgt":'float', "maxt_n":'float', "mint_n":'float', "avgt_n":'float'}, errors='ignore')

        melted = pd.melt(df, id_vars=['year','month','day'], value_vars=['mint', 'maxt', 'avgt', 'maxt_n','mint_n','avgt_n'], value_name='temperature')
        melted = melted.astype({"variable": 'string'}, errors='ignore')
      
        return melted

test = XMACIS(station_name)


#output = test.df.to_csv(index=False,header=True)
#with open('climate-data.csv', 'w') as outfile:
#  for lines in output:
#    outfile.write(lines)

<h2><b>Step 4</b></h2>
<h3>Run code below to plot data. This might take a while to render at first.</h3>


In [24]:
from jupyter_dash import JupyterDash
from dash import dcc, html, Output, Input, no_update
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import pandas as pd

dff = test.df
por_start = test.por_start_year
por_end = test.por_end_year
yrs = list(range(por_start,por_end+1))
dff = dff.astype({"variable": 'string'}, errors='ignore')
mon = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}
dff['mon'] = dff['month'].map(mon)

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.CYBORG])


app.layout = html.Div([
    dbc.Row(html.H6(' ')),
    dbc.Row([
        dbc.Col([
            html.H6('Select start year'),
            dcc.Dropdown(yrs, value=por_start, id='base_start',searchable=False)
        ]),
        dbc.Col([
            html.H6('Select end year'),
            dcc.Dropdown(yrs, value=por_end, id='base_end',searchable=False)
        ]),
        dbc.Col([
            html.H6('Select element to investigate'),
            dcc.Dropdown(
                options=[
                    {'label': 'Min Temp', 'value': 'mint'},
                    {'label': 'Max Temp', 'value': 'maxt'},
                    {'label': 'Avg Temp', 'value': 'avgt'}#,
                    #{'label': 'Normal Min', 'value': 'mint_n'},
                    #{'label': 'Normal Max', 'value': 'maxt_n'},
                    #{'label': 'Normal Avg', 'value': 'avgt_n'},
                    ],
                    value='maxt',id="element", searchable=False)
        ]),
        dbc.Col([
            html.H6('Year to compare'),
            dcc.Dropdown(yrs, value=2015, id='single_year',searchable=False)
        ])
    ]),
    dbc.Row(html.H6(' ')),
    dbc.Row(dcc.Graph(id="graphing"))
])


@app.callback(
    Output("graphing", "figure"),
    Input("base_start", "value"),
    Input("base_end", "value"),
    Input("element", "value"),
    Input("single_year", "value"))
def update_violin_chart(base_start,base_end,element,single_year):
  df = dff[(dff['variable'] == element)]
  nice_name = 'Minimum Temperature'
  col = 'blue'
  if element == 'maxt':
    nice_name = 'Maximum Temperature'
    col = 'red'
  if element == 'avgt':
    nice_name = 'Average Temperature'
    col = 'orange'
  df_base = df[(df['year'] >= base_start) & (df['year'] <= base_end)]
  df_year = df[df['year'] == single_year]

  fig = go.Figure()

  fig.add_trace(go.Violin(x=df_base['mon'], y=df_base['temperature'],
                          legendgroup='Yes', scalegroup='Yes', name=f'{base_start}-{base_end}',
                          side='negative',
                          line_color='gray')
              )
  fig.add_trace(go.Violin(x=df_year['mon'],
                          y=df_year['temperature'],
                          legendgroup='No', scalegroup='No', name=f'{single_year}',
                          side='positive',
                          line_color=col)
              )
  fig.update_traces(meanline_visible=True)
  fig.update_layout(violingap=0, violinmode='overlay')
  r = [-30,120]
  fig.update_yaxes(range = r)
  fig.update_layout(autotypenumbers='convert types')
  fig.update_layout(title=f'{test.full_name} {nice_name}',
                    legend_title='',
                    yaxis_title='Temperature  (F)')
  fig.update_layout(margin=dict(l=20, r=10, t=60, b=20))
  return fig

app.run_server(mode='inline', port=8030)

<IPython.core.display.Javascript object>